In [1]:
#default_exp data.schema
%load_ext autoreload
%autoreload 2

In [2]:
# export
# hide
import json
import re
import os
import requests
import urllib
from pathlib import Path
from typing import Any, Dict

import pymemri
from pymemri.data.itembase import Item

In [3]:
# export
# hide

# stores all generated schema classes
_SCHEMA: Dict[str, type] = dict()
    
def __getattr__(name):
    if name in _SCHEMA:
        return _SCHEMA[name]
    else:
        raise AttributeError(f"Schema for {name} not found.")

In [4]:
# export
# hide

def schema_init(self, **kwargs):
    super(type(self), self).__init__(**kwargs)

    for p in self._property_kwargs:
        setattr(self, p, kwargs.get(p, None))
        
    for e in self._edge_kwargs:
        setattr(self, e, kwargs.get(e, list()))
    
class SchemaFactory:
    """
    Class factory for schema types. Generates a schema type from a given json file or url
    """
    
    @staticmethod
    def is_valid_schema_name(name: str) -> bool:
        """Check if name is a valid schema name"""
        return re.match(r"[A-Z][a-zA-Z0-9_]*", name)

    @staticmethod
    def create_schema(name: str, properties: list, edges: list, 
                      base: type=Item, description=None) -> type:
        """
        Create a type for schema from a name, list of properties and list of edges. Optionally set the base schema.
        """
        if not  SchemaFactory.is_valid_schema_name(name):
            raise ValueError(f"{name} is not a valid schema name.")

        properties = [p for p in properties if p not in base.properties]
        edges = [e for e in edges if e not in base.edges]
        
        # Store property_kwargs and edge_kwargs of self to set these in the init.
        cls_dict = {
            "_property_kwargs": properties, 
            "_edge_kwargs": edges,
            "properties": base.properties + properties,
            "edges": base.edges + edges,
            "description": description,
            "__init__": schema_init
        }
        
        schema_cls = type(
            name, 
            (base,), 
            cls_dict
        )
        return schema_cls
    
    @staticmethod
    def from_dict(name: str, schema_dict: dict) -> type:
        """
        Creates a schema type from a dictionary. See https://gitlab.memri.io/memri/schema/ for format.
        
        TODO add edge constraints ("sequenced" and "singular")
        TODO add option for different base class
        """
        properties = schema_dict.get("properties", [])
        edges = list(schema_dict.get("relations", {}).keys())
        description=schema_dict.get("description", None)
        
        return SchemaFactory.create_schema(
            name=name,
            properties=properties,
            edges=edges,
            description=description
        )
        
    @staticmethod
    def from_file(file_name: str) -> type:
        """
        Create a schema class from json file.
        """
        schema_name = os.path.splitext(os.path.basename(file_name))[0]
        with open(file_name, "r") as f:
            schema_dict = json.load(f)
        return SchemaFactory.from_dict(schema_name, schema_dict)
    
    @staticmethod
    def from_url(url: str) -> type:
        """
        For testing, import directly from schema repository.
        """
        file_name = urllib.parse.urlparse(url)[2].rpartition('/')[-1]
        schema_name = os.path.splitext(file_name)[0]
        schema_dict = requests.get(url).json()
        return SchemaFactory.from_dict(schema_name, schema_dict)

In [5]:
# export
# hide

def get_available_schema():
    base_path = Path(pymemri.data.__file__).parent
    schema_path = base_path / "schema" / "TypeHierarchy" / "Item"
    schema_files = schema_path.rglob("*.json")
    return schema_files

def generate_schema():
    global _SCHEMA
    schema_files = get_available_schema()
    for schema_file in schema_files:
        schema_cls = SchemaFactory.from_file(schema_file)
        schema_name = schema_cls.__name__
        _SCHEMA[schema_name] = schema_cls

generate_schema()

In [6]:
# export
# hide

def get_constructor(_type, plugin_class=None, plugin_package=None, extra=None):
    import pymemri.indexers as models
    from pymemri.data.photo import IPhoto
    from pymemri.indexers.indexer import IndexerBase
    import pymemri.integrator_registry

    if _type == "Indexer" and plugin_class is not None and hasattr(pymemri.integrator_registry, plugin_class):
        return getattr(pymemri.integrator_registry, plugin_class)
    if plugin_class is not None and plugin_package is not None:
        try:
            mod = __import__(plugin_package, fromlist=[plugin_class])
            dynamic = {plugin_class: getattr(mod, plugin_class)}
        except Exception as e:
            print(f"Could not import {plugin_class}.{plugin_package}")
    else:
        dynamic = dict()

    classes = z = {**globals(), **locals(), **extra, **dynamic}
    if _type in classes:
        if _type == "Indexer":
            constructor = classes[plugin_class]
        else:
            i_class = "I" + _type
            if i_class in classes:
                constructor = classes[i_class]
            else:
                constructor = classes[_type]
    else:
        raise TypeError(f"Could not initialize item, type {_type} not registered in PodClient")
    return constructor

In [7]:
from pymemri.pod.client import PodClient
pod_client = PodClient()

In [8]:
print("Number of generated Items:", len(_SCHEMA))

assert len(_SCHEMA)

Number of generated Items: 94


In [9]:
from pymemri.data.schema import Account

test_account = Account(handle="Test Account", displayName="Test Account")

assert pod_client.add_to_schema(Account(handle="", displayName=""))
assert pod_client.create(test_account)

In [10]:
account_json = """
{
  "description": "An account or subscription, for instance for some online service, or a bank account or wallet.",
  "properties": [
    "handle",
    "displayName",
    "service",
    "itemType",
    "avatarUrl"
  ],
  "relations": {
    "belongsTo": {
      "sequenced": false,
      "singular": false
    },
    "price": {
      "sequenced": false,
      "singular": false
    },
    "location": {
      "sequenced": false,
      "singular": false
    },
    "organization": {
      "sequenced": false,
      "singular": false
    },
    "contact": {
      "sequenced": false,
      "singular": false
    }
  },
  "foregroundColor": "#ffffff",
  "backgroundColor": "#93c47d"
}
""" 

Account = SchemaFactory.from_dict("Account", json.loads(account_json))
test_account = Account(handle="Test Account", displayName="Test Account")

assert pod_client.add_to_schema(Account(handle="", displayName=""))
assert pod_client.create(test_account)

In [11]:
url = 'https://gitlab.memri.io/memri/schema/-/raw/dev/TypeHierarchy/Item/Account/Account.json'
Account = SchemaFactory.from_url(url)
test_account = Account(handle="Test Account", displayName="Test Account")

assert pod_client.add_to_schema(Account(handle="", displayName=""))
assert pod_client.create(test_account)

In [1]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted data.photo.ipynb.
Converted data.schema.ipynb.
Converted importers.Importer.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.schema.ipynb.
Converted plugin.stateful.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
